In [1]:
%cd /home/aoq559/dev/transformer/eap/edge-attribution-patching

from IPython import get_ipython
ipython = get_ipython()
if ipython is not None:
    ipython.magic("%load_ext autoreload")
    ipython.magic("%autoreload 2")

import torch

import torch as t
from torch import Tensor
import einops

from transformer_lens import HookedTransformer
from transformers import AutoTokenizer
import numpy as np

from eapp.eap_wrapper import EAP

from jaxtyping import Float

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/home/aoq559/dev/transformer/eap/edge-attribution-patching


/tmp/ipykernel_44586/3747139492.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%load_ext autoreload")
/tmp/ipykernel_44586/3747139492.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%autoreload 2")


Device: cuda


# Model Setup

In [4]:
# model = HookedTransformer.from_pretrained(
#     'EleutherAI/pythia-12b-deduped-v0',
#     center_writing_weights=False,
#     center_unembed=False,
#     fold_ln=False,
#     device=device,
#     n_devices=7,
#     move_to_device=True,
#     dtype='float16'
# )
model = HookedTransformer.from_pretrained(
    'gpt-neo-125M',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
    n_devices=5,
    move_to_device=True,
    dtype='float16'
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')
print(f"Using tokenizer {tokenizer}")

Loaded pretrained model gpt-neo-125M into HookedTransformer
Using tokenizer GPT2TokenizerFast(name_or_path='EleutherAI/gpt-neo-125m', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [5]:
# load data
import yaml
import pickle
import os
class DotDict(dict):
    """ Dot notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
yaml_file_path = "./conf/config.yaml"
with open(yaml_file_path, "r") as f:
    args = DotDict(yaml.safe_load(f))

file_name = args.data_dir
file_name += '/' + str(args.model)
file_name += '/intervention_' + str(args.n_shots) + '_shots_max_' + str(args.max_n) + '_' + args.representation
file_name += '_further_templates' if args.extended_templates else ''
file_name += '.pkl'
print(file_name)

with open(file_name, 'rb') as f:
    intervention_list = pickle.load(f)
print("Loaded data from", file_name)
if args.debug_run:
    intervention_list = intervention_list[:2]

/shared-network/shared/2024_ml_master/data/EleutherAI/gpt-neo-125m/intervention_1_shots_max_20_arabic_further_templates.pkl


Loaded data from /shared-network/shared/2024_ml_master/data/EleutherAI/gpt-neo-125m/intervention_1_shots_max_20_arabic_further_templates.pkl


In [14]:
clean_dataset.toks

tensor([[ 2215, 12313,   290, 12091,  1392,   257, 26906,   379,   262,  3650,
            11, 12091,  3066,   284,  1577,   340,   284, 12313, 50256, 50256,
         50256],
        [ 2215, 18501,   290,  8049,  1392,   257, 38987,   379,   262, 11376,
            11, 18501,  3066,   284,  1577,   340,   284,  8049, 50256, 50256,
         50256],
        [ 2215, 12246,   290,  4422,  1392,   257,  4144,   379,   262,  3650,
            11,  4422,  3066,   284,  1577,   340,   284, 12246, 50256, 50256,
         50256],
        [ 6423,    11, 17352,   290, 12969,   550,   257,   890,  4578,    11,
           290, 12979, 17352,   531,   284, 12969, 50256, 50256, 50256, 50256,
         50256],
        [ 6423,    11, 11232,   290,  7939,   547,  1762,   379,   262,  1524,
            13,  7939,  3066,   284,  1577,   257, 38987,   284, 11232, 50256,
         50256],
        [ 3260,  9005,   290,  7806,  1816,   284,   262,  4429,    11,  7806,
          2921,   257,  3644,   284,  9005, 50

# Dataset Setup

In [12]:
from ioi_dataset import IOIDataset, format_prompt, make_table
N = 25
clean_dataset = IOIDataset(
    prompt_type='mixed',
    N=N,
    tokenizer=model.tokenizer,
    prepend_bos=False,
    seed=1,
    device=device
)
corr_dataset = clean_dataset.gen_flipped_prompts('ABC->XYZ, BAB->XYZ')

make_table(
  colnames = ["IOI prompt", "IOI subj", "IOI indirect obj", "ABC prompt"],
  cols = [
    map(format_prompt, clean_dataset.sentences),
    model.to_string(clean_dataset.s_tokenIDs).split(),
    model.to_string(clean_dataset.io_tokenIDs).split(),
    map(format_prompt, clean_dataset.sentences),
  ],
  title = "Sentences from IOI vs ABC distribution",
)

                                      Sentences from IOI vs ABC distribution                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IOI prompt                              ┃ IOI subj ┃ IOI indirect obj ┃ ABC prompt                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ When Victoria and Jane got a snack at   │ Jane     │ Victoria         │ When Victoria and Jane got a snack at   │
│ the store, Jane decided to give it to   │          │                  │ the store, Jane decided to give it to   │
│ Victoria                                │          │                  │ Victoria                                │
│                                         │          │                  │                                         │
│ When Sullivan and Rose got a necklace   │ Sullivan │ Rose             │ When Sullivan and Rose got a necklace   │
│ at the garden, Sullivan decided to give │          │                  │ at the garden, Sullivan decided to give │
│ it to Rose                              │          │                  │ it to Rose                              │
│                                         │          │                  │                                         │
│ When Alan and Alex got a drink at the   │ Alex     │ Alan             │ When Alan and Alex got a drink at the   │
│ store, Alex decided to give it to Alan  │          │                  │ store, Alex decided to give it to Alan  │
│                                         │          │                  │                                         │
│ Then, Jessica and Crystal had a long    │ Jessica  │ Crystal          │ Then, Jessica and Crystal had a long    │
│ argument, and afterwards Jessica said   │          │                  │ argument, and afterwards Jessica said   │
│ to Crystal                              │          │                  │ to Crystal                              │
│                                         │          │                  │                                         │
│ Then, Jonathan and Kevin were working   │ Kevin    │ Jonathan         │ Then, Jonathan and Kevin were working   │
│ at the school. Kevin decided to give a  │          │                  │ at the school. Kevin decided to give a  │
│ necklace to Jonathan                    │          │                  │ necklace to Jonathan                    │
│                                         │          │                  │                                         │
└─────────────────────────────────────────┴──────────┴──────────────────┴─────────────────────────────────────────┘

# Metric Setup

In [6]:
intervention = intervention_list[0]
print(intervention.res_base_tok)
intervention.__dict__

[1315]


{'op3_pos': 18,
 'operator_word': None,
 'operands_alt': '2 7 6',
 'operands_base': '2 7 6',
 'operator_pos': None,
 'op2_pos': 16,
 'op1_pos': 14,
 'res_alt_tok': [1315],
 'res_base_tok': [1315],
 'res_string': None,
 'res_base_string': '15',
 'res_alt_string': '15',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': 'The result of 2 + 7 + 6 =',
 'alt_string': 'The result of 2 + 7 + 6 =',
 'few_shots': 'The result of 7 + 5 + 3 = 15. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y} + {z})',
 'enc': GPT2Tokenizer(name_or_path='EleutherAI/gpt-neo-125m', vocab_size=50257, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=Tr

In [27]:
intervention = intervention_list[0]
with torch.no_grad():
    clean_logits = model(intervention.base_string_tok).cpu().numpy()
    print(clean_logits)
    corrupt_logits = model(intervention.alt_string_tok).cpu().numpy()
    clean_logits_argmax = np.argmax(clean_logits, axis=2)[0, -1]
    print(clean_logits_argmax)
    clean_logit = clean_logits[:, -1, intervention.res_base_tok[0]]
    corrupt_logit = corrupt_logits[:, -1, intervention.pred_res_alt_tok[0]]
    logit_diff = corrupt_logit - clean_logit
    print(clean_logit)
    print(corrupt_logit)

    
    # clean_logit_argmax = torch.argmax(clean_logits, dim=2)
    # corrupt_logit_argmax = torch.argmax(corrupt_logits, dim=2)
    # print(clean_logit_argmax)
    # next_word_index = clean_logit_argmax[0][-1]
    # next_word = model.tokenizer.convert_ids_to_tokens(next_word_index.item())
    # print(next_word)
    
    # clean_prediction = torch.argmax(clean_logits, dim=2)
    # print(clean_prediction)
    # next_word_index = clean_prediction[0][-1]
    # next_word = tokenizer.convert_ids_to_tokens(next_word_index.item())
    # print(next_word_index)
    # print(next_word)
    # print(torch.argmax(clean_logits[:, -1, :]))
    # print(corrupt_logits.shape)

[[[ -7.465  -6.3    -8.26  ... -13.19   -9.43   -6.61 ]
  [ -7.74  -10.9   -15.34  ... -20.4   -18.86  -14.484]
  [-12.34  -10.46  -13.24  ... -14.28  -12.445 -10.54 ]
  ...
  [-13.74  -12.58  -14.51  ... -22.11  -15.08  -10.7  ]
  [-11.086 -12.234 -14.95  ... -23.89  -17.08  -11.43 ]
  [-11.68  -11.29  -13.55  ... -18.84  -12.17   -7.54 ]]]
1315
[-0.77]
[0.03464]


In [30]:
# Create an empty list to store the stacked tensors
stacked = []
# Iterate over each intervention in intervention_list
for intervention in intervention_list:
    # Vertically stack the 2-D tensor and append it to the list
    stacked.append(intervention.res_base_tok[0])
print(stacked)

[1315, 1467]


In [ ]:
# Create an empty list to store the stacked tensors
stacked_tensors = []
# Iterate over each intervention in intervention_list
for intervention in intervention_list:
    # Vertically stack the 2-D tensor and append it to the list
    stacked_tensors.append(intervention.base_string_tok.to(device))
# Convert the list of stacked tensors into a single tensor
dataset = torch.vstack(stacked_tensors)

In [45]:
import intervention_dataset
intervention_data = intervention_dataset.InterventionDataset(intervention_list, device, model.tokenizer)
intervention_data.create_intervention_dataset()
print(intervention_data.pred_res_alt_toks[:2])

[807, 513]


In [54]:
with t.no_grad():
    clean_logits = model(dataset)
print(clean_logits)
batch_size = clean_logits.size(0)
print(batch_size)
clean_logits = clean_logits[range(batch_size), -1, [-1, -1]]
print(clean_logits)

tensor([[[ -7.4648,  -6.3008,  -8.2578,  ..., -13.1875,  -9.4297,  -6.6094],
         [ -7.7383, -10.8984, -15.3438,  ..., -20.4062, -18.8594, -14.4844],
         [-12.3438, -10.4609, -13.2422,  ..., -14.2812, -12.4453, -10.5391],
         ...,
         [-13.7422, -12.5781, -14.5078,  ..., -22.1094, -15.0781, -10.7031],
         [-11.0859, -12.2344, -14.9531,  ..., -23.8906, -17.0781, -11.4297],
         [-11.6797, -11.2891, -13.5469,  ..., -18.8438, -12.1719,  -7.5391]],

        [[ -7.4648,  -6.3008,  -8.2578,  ..., -13.1875,  -9.4297,  -6.6094],
         [ -7.7383, -10.8984, -15.3438,  ..., -20.4062, -18.8594, -14.4844],
         [-12.3438, -10.4609, -13.2422,  ..., -14.2812, -12.4453, -10.5391],
         ...,
         [-13.4297, -12.4141, -15.0938,  ..., -22.2812, -15.5703, -10.2109],
         [-10.4844, -12.4688, -15.3672,  ..., -23.6875, -17.6875, -11.6172],
         [-11.4219, -10.7031, -14.1172,  ..., -18.4844, -12.7969,  -8.2266]]],
       device='cuda:5', dtype=torch.float16)

In [55]:
def ave_logit_difference(
    logits: Float[Tensor, 'batch seq d_vocab'],
    intervention_dataset,
    per_prompt: bool = False
):
    batch_size = logits.size(0)
    clean_logits = logits[range(batch_size), -1, intervention_dataset.res_base_toks[:batch_size]].cpu().numpy()
    corrupt_logits = logits[range(batch_size), -1, intervention_dataset.pred_res_alt_toks[:batch_size]].cpu().numpy()
    logit_diff = corrupt_logits - clean_logits
    return logit_diff if per_prompt else logit_diff.mean()
    
    
with t.no_grad():
    clean_logits = model(intervention_data.base_string_toks)
    corrupt_logits = model(intervention_data.alt_string_toks)
    clean_logit_diff = ave_logit_difference(clean_logits, intervention_data, per_prompt=False)
    corrupt_logit_diff = ave_logit_difference(corrupt_logits, intervention_data, per_prompt=False)
print(clean_logit_diff)
print(corrupt_logit_diff)

def metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    intervention_dataset: intervention_dataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_difference(logits, intervention_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)


# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

-1.699
1.229


In [19]:
dataset

tensor([[ 464, 1255,  286,  767, 1343,  642, 1343,  513,  796, 1315,   13,  383,
         1255,  286,  362, 1343,  767, 1343,  718,  796],
        [ 464, 1255,  286,  642, 1343,  642, 1343,  718,  796, 1467,   13,  383,
         1255,  286,  513, 1343,  604, 1343,  860,  796]], device='cuda:0')

In [32]:
clean_logits[:, -1, :].argmax()

198

In [8]:
#tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-12b-deduped-v0')
print(tokenizer.decode(intervention.res_base_tok))
print(intervention.res_base_tok)
tokenizer.decode(intervention.base_string_tok[0])

 15
[1315]


'The result of 7 + 5 + 3 = 15. The result of 2 + 7 + 6 ='

In [7]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    ioi_dataset: IOIDataset,
    per_prompt: bool = False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    # Get logits for indirect objects
    batch_size = logits.size(0)
    # logits[batch_size, last_position, ID of IO]
    io_logits = logits[range(batch_size), ioi_dataset.word_idx['end'][:batch_size], ioi_dataset.io_tokenIDs[:batch_size]]
    s_logits = logits[range(batch_size), ioi_dataset.word_idx['end'][:batch_size], ioi_dataset.s_tokenIDs[:batch_size]]
    # Get logits for subject
    logit_diff = io_logits - s_logits
    return logit_diff if per_prompt else logit_diff.mean()

with t.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(clean_logits, clean_dataset).item() # logit difference for clean run between correct and incorrect answer
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item() # logit difference for corrupt run between correct and incorrect answer

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    ioi_dataset: IOIDataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_diff(logits, ioi_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def negative_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -ioi_metric(logits)
    
# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

Clean direction: 2.805162191390991, Corrupt direction: 1.4692586660385132
Clean metric: 1.0, Corrupt metric: 0.0


# Run Experiment

In [10]:
model.reset_hooks()

graph = EAP(
    model,
    clean_dataset.toks,
    corr_dataset.toks,
    ioi_metric,
    upstream_nodes=["mlp", "head"],
    downstream_nodes=["mlp", "head"],
    batch_size=25
)

top_edges = graph.top_edges(n=10, abs_scores=True)
for from_edge, to_edge, score in top_edges:
    print(f'{from_edge} -> [{round(score, 3)}] -> {to_edge}')

Saving activations requires 0.0004 GB of memory per token


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.42s/it]


head.9.9 -> [-0.029] -> head.11.10.q
head.10.7 -> [0.029] -> head.11.10.q
head.5.5 -> [0.021] -> head.8.6.v
head.9.9 -> [-0.019] -> head.10.7.q
head.5.5 -> [-0.019] -> mlp.5
mlp.0 -> [0.018] -> head.6.9.q
mlp.0 -> [-0.015] -> mlp.4
head.5.5 -> [-0.015] -> head.6.9.q
mlp.0 -> [-0.014] -> head.11.10.k
head.3.0 -> [-0.014] -> mlp.5


In [13]:
top_edges

[('head.9.9', 'head.11.10.q', -0.0291709266602993),
 ('head.10.7', 'head.11.10.q', 0.02903885766863823),
 ('head.5.5', 'head.8.6.v', 0.020961817353963852),
 ('head.9.9', 'head.10.7.q', -0.01940404810011387),
 ('head.5.5', 'mlp.5', -0.018816370517015457),
 ('mlp.0', 'head.6.9.q', 0.01845172606408596),
 ('mlp.0', 'mlp.4', -0.0152858542278409),
 ('head.5.5', 'head.6.9.q', -0.014597196131944656),
 ('mlp.0', 'head.11.10.k', -0.013949546962976456),
 ('head.3.0', 'mlp.5', -0.013543691486120224)]